In [1]:
import pandas as pd
import json

Here we open the json data and normalize the data such that each row corresponds to one ingredient.

The dataset has already been parsed by a CRF model to turn the freeform recipe ingredients into structured data (name, unit, quantity).

In [4]:
with open('enriched_recipes.json') as f:
    data = json.load(f)
    df = pd.json_normalize(
        data, 'parsed_ingredients',
        meta=['author',
              'photo_url',
              'prep_time_minutes',
              'rating_stars',
              'review_count',
              'title',
              'total_time_minutes',
              'url'
             ],
        record_prefix='ingredient_', errors='ignore')

The output data looks like this:

In [7]:
df

,ingredient_comment,ingredient_input,ingredient_name,ingredient_other,ingredient_qty,ingredient_unit,ingredient_range_end,author,photo_url,prep_time_minutes,rating_stars,review_count,title,total_time_minutes,url
0,chilled and cubed,"1/2 cup unsalted butter, chilled and cubed",unsalted butter,",",1/2,cup,NaN,Stephanie,http://images.media-allrecipes.com/userphotos/...,55,4.32,46,"Basil, Roasted Peppers and Monterey Jack Cornb...",100,http://allrecipes.com/Recipe/6664/
1,chopped,1 cup chopped onion,onion,NaN,1,cup,NaN,Stephanie,http://images.media-allrecipes.com/userphotos/...,55,4.32,46,"Basil, Roasted Peppers and Monterey Jack Cornb...",100,http://allrecipes.com/Recipe/6664/
2,NaN,1 3/4 cups cornmeal,cornmeal,NaN,1 3/4,cup,NaN,Stephanie,http://images.media-allrecipes.com/userphotos/...,55,4.32,46,"Basil, Roasted Peppers and Monterey Jack Cornb...",100,http://allrecipes.com/Recipe/6664/
3,NaN,1 1/4 cups all-purpose flour,all-purpose flour,NaN,1 1/4,cup,NaN,Stephanie,http://images.media-allrecipes.com/userphotos/...,55,4.32,46,"Basil, Roasted Peppers and Monterey Jack Cornb...",100,http://allrecipes.com/Recipe/6664/
4,NaN,1/4 cup white sugar,white sugar,NaN,1/4,cup,NaN,Stephanie,http://images.media-allrecipes.com/userphotos/...,55,4.32,46,"Basil, Roasted Peppers and Monterey Jack Cornb...",100,http://allrecipes.com/Recipe/6664/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836531,NaN,2 tablespoons brown sugar,brown sugar,NaN,2,tablespoon,NaN,larkspur,http://images.media-allrecipes.com/global/reci...,32,0.0,0,Blueberry Oatmeal Cream Brulee,44,http://allrecipes.com/Recipe/258791/
836532,NaN,2 tablespoons maple syrup,maple syrup,NaN,2,tablespoon,NaN,larkspur,http://images.media-allrecipes.com/global/reci...,32,0.0,0,Blueberry Oatmeal Cream Brulee,44,http://allrecipes.com/Recipe/258791/
836533,NaN,2 tablespoons butter,butter,NaN,2,tablespoon,NaN,larkspur,http://images.media-allrecipes.com/global/reci...,32,0.0,0,Blueberry Oatmeal Cream Brulee,44,http://allrecipes.com/Recipe/258791/
836534,NaN,1 cup blueberries,blueberries,NaN,1,cup,NaN,larkspur,http://images.media-allrecipes.com/global/reci...,32,0.0,0,Blueberry Oatmeal Cream Brulee,44,http://allrecipes.com/Recipe/258791/


Filter the data where recipe ratings are valid (more than 20 reviews and more than 0 stars).

Then, group the ingredients by ingredient name, and average each ingredient's numerical rating.

In [6]:
grouped = df.where(df['rating_stars'] > 0)
            .where(df['review_count'] > 20)
            .groupby(['ingredient_name'])['rating_stars']
            .mean()

In [5]:
grouped

ingredient_name
(.13 ounce) envelope unsweetened grape soft drink mix     4.400000
(.13 ounce) envelope unsweetened orange soft drink mix    4.400000
(.14 ounce) package sugar free instant lemonade powder    4.490000
(.18 ounce) packet sazon seasoning                        4.520000
(.25 ounce) envelope active dry yeast                     4.471250
                                                            ...   
zucchini chunks                                           4.560000
zucchini matchsticks                                      4.540000
zucchini squash                                           4.454000
zucchini squashes                                         4.260000
zucchinis                                                 4.426111
Name: rating_stars, Length: 9297, dtype: float64

In [18]:
spices = [
    'allspice'
    'angelica'
    'anise'
    'asafoetida'
    'bay leaf'
    'basil'
    'bergamot'
    'black cumin'
    'black mustard'
    'black pepper'
    'borage'
    'brown mustard'
    'burnet'
    'caraway'
    'cardamom'
    'cassia'
    'catnip'
    'cayenne pepper'
    'celery seed'
    'chervil'
    'chicory'
    'chili pepper'
    'chives'
    'cicely'
    'cilantro'
    'cinnamon'
    'clove'
    'coriander'
    'costmary'
    'cumin'
    'curry'
    'dill'
    'fennel'
    'fenugreek'
    'filé'
    'ginger'
    'grains of paradise'
    'holy basil'
    'horehound'
    'horseradish'
    'hyssop'
    'lavender'
    'lemon balm'
    'lemon grass'
    'lemon verbena'
    'licorice'
    'lovage'
    'mace'
    'marjoram'
    'nutmeg'
    'oregano'
    'paprika'
    'parsley'
    'peppermint'
    'poppy seed'
    'rosemary'
    'rue'
    'saffron'
    'sage'
    'savory'
    'sesame'
    'sorrel'
    'star anise'
    'spearmint'
    'tarragon'
    'thyme'
    'turmeric'
    'vanilla'
    'wasabi'
    'white mustard'
]

In [25]:
spice_grouped = df[df['ingredient_name'].isin(spices)]

In [27]:
df['ingredient_name']

0           unsalted butter
1                     onion
2                  cornmeal
3         all-purpose flour
4               white sugar
                ...        
836531          brown sugar
836532          maple syrup
836533               butter
836534          blueberries
836535          white sugar
Name: ingredient_name, Length: 836536, dtype: object